## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
retail = pd.read_csv('/kaggle/input/online-retail-ii-uci/online_retail_II.csv')

### Checking and Preparing the Data

In [3]:
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


**First Observations:**
- 'InvoiceDate' column contains duplicate information but they are of type object when they are of type datetime. We need to convert this.
- Most columns have 1067371 rows except for 'Description' which has 1062989 rows and 'Customer ID' has 824364 rows. We don't particularly require the 'Description' column but we do need the 'Customer ID' column. We have to drop every row that has a null 'Customer ID'.
- 'Customer ID' column is of type float64 when it is sufficient to convert them into int64.
- While we have columns for 'Price' and 'Quantity' it would be helpful to get the Total Price for each row.
- Finally, we will sort the data chronologically.

In [5]:
retail['InvoiceDate_DT'] = pd.to_datetime(retail['InvoiceDate'])
retail = retail[retail['Customer ID'].notna()]
retail['Customer ID'] = retail['Customer ID'].astype(np.int64)
retail['Total'] = retail['Quantity'] * retail['Price']
retail = retail.sort_values('InvoiceDate_DT')

Let's take a look at the values.

In [6]:
retail

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,InvoiceDate_DT,Total
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,2009-12-01 07:45:00,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,2009-12-01 07:45:00,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,2009-12-01 07:45:00,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,2009-12-01 07:45:00,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,2009-12-01 07:45:00,30.00
...,...,...,...,...,...,...,...,...,...,...
1067357,581587,22555,PLASTERS IN TIN STRONGMAN,12,2011-12-09 12:50:00,1.65,12680,France,2011-12-09 12:50:00,19.80
1067356,581587,22556,PLASTERS IN TIN CIRCUS PARADE,12,2011-12-09 12:50:00,1.65,12680,France,2011-12-09 12:50:00,19.80
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680,France,2011-12-09 12:50:00,14.85
1067361,581587,22730,ALARM CLOCK BAKELIKE IVORY,4,2011-12-09 12:50:00,3.75,12680,France,2011-12-09 12:50:00,15.00


## General Sale Analytics
### Most and Least Expensive Product
Let us look at the products that have the highest and lowest price.

In [7]:
most_expensive = retail.loc[retail['Price'] == retail['Price'].max()]
most_expensive

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,InvoiceDate_DT,Total
748142,C556445,M,Manual,-1,2011-06-10 15:31:00,38970.0,15098,United Kingdom,2011-06-10 15:31:00,-38970.0


In [8]:
least_expensive = retail.loc[retail['Price'] == retail['Price'].min()]
least_expensive = least_expensive.groupby(['Description', 'Price'], 
                                          as_index = False)['Quantity'].agg('sum')
least_expensive

,Description,Price,Quantity
0,FLAMINGO LIGHTS,0.0,24
1,OVAL WALL MIRROR DIAMANTE,0.0,1
2,36 FOIL STAR CAKE CASES,0.0,144
3,6 RIBBONS EMPIRE,0.0,12
4,ADVENT CALENDAR GINGHAM SACK,0.0,4
...,...,...,...
56,SET/5 RED SPOTTY LID GLASS BOWLS,0.0,2
57,TV DINNER TRAY DOLLY GIRL,0.0,9
58,This is a test product.,0.0,10
59,VINTAGE CREAM CAT FOOD CONTAINER,0.0,1


We can see that the most expensive product in the dataset is priced at £ 38,970. It is interesting that the quantity is -1 meaning that someone returned the product or they have not paid for it.

For finding the least expensive product we are going to exclude the 0 values and find the least expensive product after that.

In [9]:
retail_non_zero_price = retail.loc[retail['Price'] != 0]
least_expensive_not_null = retail_non_zero_price.loc[retail_non_zero_price['Price'] 
                                                     == retail_non_zero_price['Price'].min()]
least_expensive_not_null = least_expensive_not_null.groupby(['Description', 'Price'], 
                                                           as_index = False)['Quantity'].agg('sum')
least_expensive_not_null

,Description,Price,Quantity
0,Bank Charges,0.001,1
1,PADS TO MATCH ALL CUSHIONS,0.001,17


The cheapest product is 'PADS TO MATCH ALL CUSHIONS'. It is priced at £0.001. This means you can get 100 of them for £1.
Bank charges are alo priced at £0.001

### Customer Analysis: Total Purchase Amount
Next, I'm interested in finding out, which 10 customers have spent the most in the online shop for the two years, and thus are the most valuable clients. Also, I'll define the 10 customers that are spending the least.

In [10]:
retail_customers = retail.groupby(['Customer ID', 'Country'], as_index = False)['Total'].agg('sum')
retail_customers_asc = retail_customers.sort_values('Total').head(10)
retail_customers_dsc = retail_customers.sort_values('Total', ascending = False).head(10)

In [11]:
fig = go.Figure(data = [
    go.Bar(name = 'Customer with Max Total Purchase Amount',
          x = retail_customers_dsc['Customer ID'].astype(str),
          y = retail_customers_dsc['Total'],
          marker_opacity = 1,
          hovertext = retail_customers_dsc['Country'],
          marker = {'color': retail_customers_dsc['Total'],
                    'colorscale': 'Rainbow'})
])
fig.update_traces(texttemplate = '£%{y:.3s}', textposition = 'inside')

fig.update_layout(title = 'Customers with Max Total Purchase Amount',
                  title_x = 0.45,
                  xaxis_title = 'Customer ID',
                  yaxis_title = 'Total Amount, £',
                  plot_bgcolor = 'white')
fig.show()

In [12]:
fig = go.Figure(data = [
    go.Bar(name = 'Customers with Min Total Purchase Amount',
           x = retail_customers_asc['Customer ID'].astype(str),
           y = retail_customers_asc['Total'],
           marker_opacity = 1,
           hovertext = retail_customers_asc['Country'],
           marker = {'color': retail_customers_asc['Total'],
                     'colorscale': 'Rainbow'}
          )
])

fig.update_xaxes(side = "top")

fig.update_traces(texttemplate='£%{y:.3s}', textposition = 'inside')

fig.update_layout(title = 'Customers with Min Total Purchase Amount',
                  title_x = 0.45,
                  title_y = 0.97,
                  xaxis_title = 'Customer ID',
                  yaxis_title = 'Total Amount Returned, £',
                  plot_bgcolor='white')
fig.show()

Customer 1802 has spent the most amount of money at £598K in the United Kingdom. This customer is followed by customer 14646 at £523K in Netherlands. In general most of the top customers come from countries with mature economies like United Kingdom, Netherlands, EIRE etc.

The largest negative sum is £25K and the customer with this returned amount comes from the United Kingdom. 

### Countries Analysis: Total Purchase Amount
I will find the 10 countries where the total purchase amount was the largest and the 10 countries where the total purchase amount was the smallest.

In [13]:
retail_countries_purchase = retail.groupby(['Country'], as_index = False)['Total'].agg('sum')
retail_countries_purchase_asc = retail_countries_purchase.sort_values('Total').head(10)
retail_countries_purchase_dsc = retail_countries_purchase.sort_values('Total', ascending = False).head(10)

In [14]:
fig = go.Figure(data = [
    go.Bar(name = 'Countries with Max Total Purchase Amount',
           x = retail_countries_purchase_dsc['Country'].astype(str),
           y = retail_countries_purchase_dsc['Total'],
           marker_opacity = 1,
           marker = {'color': retail_countries_purchase_dsc['Total'],
                     'colorscale': 'Rainbow'}
          )
])

fig.update_traces(texttemplate='£%{y:.3s}', textposition = 'outside')

fig.update_layout(title = 'Countries with Max Total Purchase Amount',
                  title_x = 0.45,
                  xaxis_title = 'Countries',
                  yaxis_title = 'Total Amount, £',
                  plot_bgcolor='white')
fig.show()

In [15]:
fig = go.Figure(data = [
    go.Bar(name = 'Countries with Min Total Purchase Amount',
           x = retail_countries_purchase_asc['Country'].astype(str),
           y = retail_countries_purchase_asc['Total'],
           marker_opacity = 1,
           marker = {'color': retail_countries_purchase_asc['Total'],
                     'colorscale': 'Rainbow'}
          )
])

fig.update_traces(texttemplate='£%{y:.3s}', textposition = 'inside')

fig.update_layout(title = 'Countries with Min Total Purchase Amount',
                  title_x = 0.45,
                  title_y = 0.97,
                  xaxis_title = 'Countries',
                  yaxis_title = 'Total Amount, £',
                  plot_bgcolor='white')
fig.show()

If we look at the leaders for maximal amount spent we will see that the United Kingdom is the sole leader with a purchase amount of £13.8M. The second closest amount is £579K from EIRE countries. All the countries in the top ten are located in Europe.

The country with the smallest value is Saudi Arabia who have only spent £131 in two years. While generally countries in Europe belong to the top 10 in purchasing amount Czech Republic seems to be in the bottom 10 in total purchasing amount.

### Country Analysis: Unique Customer Numbers
I am going to find the top 10 and bottom 10 countries that has the most and least unique customer numbers.

In [16]:
retail_countries_customers = retail.groupby(['Country'], as_index = False)['Customer ID'].agg({'Customer ID': 'nunique'})
retail_countries_customers.rename(columns = {'Customer ID': 'Customer Count'}, inplace = True)
retail_countries_customers_asc = retail_countries_customers.sort_values('Customer Count').head(10)
retail_countries_customers_dsc = retail_countries_customers.sort_values('Customer Count', ascending = False).head(10)

/tmp/ipykernel_33/595464315.py:1: FutureWarning:

Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.



In [17]:
fig = go.Figure(data = [
    go.Bar(name = 'Countries with Largest Number of Unique Customers',
           x = retail_countries_customers_dsc['Country'].astype(str),
           y = retail_countries_customers_dsc['Customer Count'],
           marker_opacity = 1,
           marker = {'color': retail_countries_customers_dsc['Customer Count'],
                     'colorscale': 'Rainbow'}
          )
])

fig.update_traces(texttemplate='£%{y:.3s}', textposition = 'outside')

fig.update_layout(title = 'Countries with Largest Number of Unique Customers',
                  title_x = 0.45,
                  xaxis_title = 'Countries',
                  yaxis_title = 'Number of Unique Customers',
                  plot_bgcolor='white')
fig.show()

In [18]:
fig = go.Figure(data = [
    go.Bar(name = 'Countries with Smallest Number of Unique Customers',
           x = retail_countries_customers_asc['Country'].astype(str),
           y = retail_countries_customers_asc['Customer Count'],
           marker_opacity = 1,
           marker = {'color': retail_countries_customers_asc['Customer Count'],
                     'colorscale': 'Rainbow'}
          )
])

fig.update_traces(texttemplate='£%{y:.3s}', textposition = 'outside')

fig.update_layout(title = 'Countries with Smallest Number of Unique Customers',
                  title_x = 0.45,
                  title_y = 0.97,
                  xaxis_title = 'Countries',
                  yaxis_title = 'Number of Unique Customers',
                  plot_bgcolor='white')
fig.show()

In UK, 5.41k of unique customers have made purchases in two years, while Germany and France follow up with 107 and 95 customers correspondingly. The difference between top position and the followers are huge.
Also there are many countries with single unique customers like Thailand, Nigeria, Iceland, Saudi Arabia etc. Here Czech Republic is an outlier in the list of Asian, South American, and African countries.

### Product Analysis: Quantity
I will find the top 10 best selling products and the top 10 returned products. I will remove products with description 'Discount" and 'CRUK Commission' to concentrate on the representative data.

In [19]:
retail_products = retail.groupby(['Description'], as_index = False)['Quantity'].agg('sum')
retail_products = retail_products.loc[~retail_products['Description'].isin(['Discount', 'CRUK Commission'])]
retail_products_asc = retail_products.sort_values('Quantity').head(10)
retail_products_asc['Quantity'] = retail_products_asc['Quantity'] * (-1)
retail_products_dsc = retail_products.sort_values('Quantity', ascending = False).head(10)

In [20]:
fig = go.Figure(data = [
    go.Bar(name = '10 Bestselling Products',
           x = retail_products_dsc['Description'].astype(str),
           y = retail_products_dsc['Quantity'],
           marker_opacity = 1,
           marker = {'color': retail_products_dsc['Quantity'],
                     'colorscale': 'Rainbow'}
          )
])

fig.update_traces(texttemplate='£%{y:.3s}', textposition = 'outside')
fig.update_layout(barmode = 'group', showlegend = False)

fig.update_layout(title = '10 Bestselling Products',
                  title_x = 0.45,
                  xaxis_title = 'Products',
                  yaxis_title = 'Total Quantity Sold',
                  plot_bgcolor='white')
fig.show()

In [21]:
fig = go.Figure(data = [
    go.Bar(name = 'Top 10 Products with most Returns',
           x = retail_products_asc['Description'].astype(str),
           y = retail_products_asc['Quantity'],
           marker_opacity = 1,
           marker = {'color': retail_products_asc['Quantity'],
                     'colorscale': 'Rainbow'}
          )
])

fig.update_traces(texttemplate='£%{y:.3s}', textposition = 'outside')
fig.update_layout(barmode = 'group', showlegend = False)

fig.update_layout(title = 'Top 10 Products with Most Returns',
                  title_x = 0.45,
                  title_y = 0.97,
                  xaxis_title = 'Products',
                  yaxis_title = 'Number Quantity Returned',
                  plot_bgcolor='white')
fig.show()

We can see that **World War II Gliders** are the clear bestsellers selling £107K in two years.
The product with the most returns is **White Cherry Lights** selling 105 of them in two years.

### Cohort Analysis
Cohort Analysis is usually used to show the behaviour of customers over time. It is incredibly useful tool to find some insights about customer experience, and to understand how to improve it.
I will use cohort analysis to investigate retention rate and average amount of products sold, and choose to use quarters of the year to define my cohort.

In [22]:
retail = retail[retail['InvoiceDate_DT'].dt.year != 2009]
retail['InvoiceQuarter'] = ('Q' + retail['InvoiceDate_DT'].dt.quarter.astype(str) + 
                            '/' + retail['InvoiceDate_DT'].dt.year.astype(str))
quarters_map = dict(zip(retail['InvoiceQuarter'].unique(),
                        range(len(retail['InvoiceQuarter'].unique()))))
retail['InvoiceQuarterID'] = retail['InvoiceQuarter'].map(quarters_map)
retail['CohortQuarterID'] = retail.groupby('Customer ID')['InvoiceQuarterID'].transform('min')
retail['CohortQuarter'] = retail['CohortQuarterID'].map(dict(zip(quarters_map.values(), quarters_map.keys())))
retail['CohortIndex'] = retail['InvoiceQuarterID'] - retail['CohortQuarterID']

/tmp/ipykernel_33/1721321885.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_33/1721321885.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_33/1721321885.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_33/1721321

#### Retention Cohort Analysis
We can perform retention analysis to investigate the number of active users. For this, I will group the data, build a pivot table, and calculate the retention rate in percentage.
Finally I will plot the triangle chart with plotly heatmap.

In [23]:
cohort_retention = retail.groupby(['CohortQuarterID', 'CohortIndex'])['Customer ID'].apply(
    pd.Series.nunique).reset_index()
cohort_retention.rename(columns = {'Customer ID': 'Customer Count'}, inplace = True)
cohort_retention_count = cohort_retention.pivot_table(index = 'CohortQuarterID',
                                                      columns = 'CohortIndex',
                                                      values = 'Customer Count')
cohort_retention_count['CohortQuarter'] = cohort_retention_count.index.map(dict(zip(quarters_map.values(), quarters_map.keys())))
cohort_retention_count = cohort_retention_count.set_index('CohortQuarter')
cohort_size = cohort_retention_count.iloc[:, 0]
retention = cohort_retention_count.divide(cohort_size, axis = 0)
retention = (retention*100).round(2)
retention = retention.iloc[::-1]

In [24]:
fig = go.Figure(data = go.Heatmap(
                       z = retention,
                       y = retention.index,
                       colorscale = 'Greens',
                       text = retention,
                       texttemplate = "%{text}%",
                       colorbar_title = 'Retention Rate',
                       xgap = 3,
                       ygap = 3
))

fig.update_xaxes(side = 'top')

fig.update_layout(title = 'Cohort Analysis: Retention Rate',
                  xaxis_title = 'Cohorts',
                  yaxis_title = 'Quarters',
                  title_x = 0.5,
                  title_y = 0.99,
                  plot_bgcolor = 'white')
fig.show()

#### Average Quantity Cohort Analysis
I am going to check how many products were sold on average, and how this changes with time.
I am going to group the products on the mean function this time.

In [25]:
cohort_quantity = retail.groupby(['CohortQuarterID', 'CohortIndex'])['Quantity'].mean().reset_index()
cohort_quantity.rename(columns = {'Quantity': 'Average Quantity'}, inplace = True)
average_quantity = cohort_quantity.pivot_table(index = 'CohortQuarterID',
                                               columns = 'CohortIndex',
                                               values = 'Average Quantity').round(1)
average_quantity['CohortQuarter'] = average_quantity.index.map(dict(zip(quarters_map.values(),
                                                                        quarters_map.keys())))
average_quantity = average_quantity.set_index('CohortQuarter')
average_quantity = average_quantity.iloc[::-1]

In [26]:
fig = go.Figure(data = go.Heatmap(
                       z = average_quantity,
                       y = average_quantity.index,
                       colorscale = 'Blues',
                       text = average_quantity,
                       texttemplate = "%{text}%",
                       colorbar_title = 'Retention Rate',
                       xgap = 3,
                       ygap = 3
))

fig.update_xaxes(side = 'top')

fig.update_layout(title = 'Cohort Analysis: Average Quantity',
                  xaxis_title = 'Cohorts',
                  yaxis_title = 'Quarters',
                  title_x = 0.5,
                  title_y = 0.99,
                  plot_bgcolor = 'white')
fig.show()

### Online Sales On Timeline
I will build two more charts to breakdown how the sales amount was changing with time. I will breakdown data by quarters and by months.

#### Quarterly Sales

In [27]:
retail_quarters_by_sales = retail.groupby(['InvoiceQuarterID'], as_index = False)['Total'].agg('sum')
retail_quarters_by_sales['InvoiceQuarter'] = retail_quarters_by_sales['InvoiceQuarterID'].map(dict(zip(quarters_map.values(),
                                                                                                       quarters_map.keys())))

In [29]:
fig = go.Figure(data = [
                go.Bar(name = 'Quarterly Sales',
                       x = retail_quarters_by_sales['InvoiceQuarter'],
                       y = retail_quarters_by_sales['Total'],
                       marker_opacity = 1,
                       text = 'Total',
                       marker = {'color': retail_quarters_by_sales['Total'],
                                 'colorscale': 'Portland'}
                      )
])

fig.update_traces(texttemplate = '%{text}: £%{y:.3s}', textposition = 'outside')
fig.update_layout(barmode = 'group', showlegend = False)

fig.update_layout(title = 'Quarterly Sales',
                  title_x = 0.45,
                  xaxis_title = 'Quarters',
                  yaxis_title = 'Total Amount, £',
                  plot_bgcolor = 'white')
fig.show()

We can see the seasonal fluctuations here. People tend to buy a lot before Christmas and a significant drop off occurs after it.

#### Monthly Sales

In [31]:
retail_monthly = retail.copy()
retail_monthly['InvoiceMonth'] = retail_monthly['InvoiceDate_DT'].dt.to_period('M')
retail_monthly_by_sales = retail_monthly.groupby(['InvoiceMonth'], as_index = False)['Total'].agg('sum')

In [33]:
fig = go.Figure(data = [
                go.Bar(name = 'Monthly Sales',
                       x = retail_monthly_by_sales['InvoiceMonth'].astype(str),
                       y = retail_monthly_by_sales['Total'],
                       marker_opacity = 1,
                       text = 'Total',
                       marker = {'color': retail_monthly_by_sales['Total'],
                                 'colorscale': 'Portland'}
                      )
])

fig.update_layout(barmode = 'group', showlegend = False)

fig.update_layout(title = 'Monthly Sales',
                  title_x = 0.45,
                  xaxis_title = 'Months',
                  yaxis_title = 'Total Amount, £',
                  plot_bgcolor = 'white')
fig.show()

With this breakdown we can easily detect the Christmas peaks. The sales for December 2011 have dropped almost 50 percent from December 2010.

### Conclusion
Let's take a moment to summarize the most impactful insights:
- The most expensive product costs £38970, while cheapest one costs £0.001.
- Customer with ID 18102, who lives in the UK has purchased goods with a total value of £598k in the last two years. This makes him the most valuable client. Another customer with the ID 17399 has returned goods with a value of £25k during the same time slot.
- United Kingdom is leading the purchases with amount of £13.8M and has the largest amount of unique customers. The country with the smallest purchase value is Saudi Arabia. Interestingly, customers from the Czech Republic tend to buy very little from the store, unlike other European countries. This might be valuable for the marketing team.
- The product 'World War II Gliders' is our top bestseller.
- Retention rate has dropped significantly in 2010, matching the drop in average sold products, but has raised again in 2011.
- Store sales are prominently seasonal, however, with December 2011 being twice less good in terms of income compared to December 2010. This would be a great starting point for deeper analysis.